In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [3]:
images_dir = "/kaggle/input/images"
labels_dir = "/kaggle/input/labelspoints"

# Target YOLOv8 folder structure
base_dir = "/kaggle/working/dataset"
os.makedirs(f"{base_dir}/images/train", exist_ok=True)
os.makedirs(f"{base_dir}/images/val", exist_ok=True)
os.makedirs(f"{base_dir}/labels/train", exist_ok=True)
os.makedirs(f"{base_dir}/labels/val", exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
train_imgs, val_imgs = train_test_split(image_files, test_size=0.2, random_state=42)

# Move images and labels to correct folders
for img_list, split in [(train_imgs, "train"), (val_imgs, "val")]:
    for img_name in img_list:
        base = os.path.splitext(img_name)[0]
        label_file = f"{base}.txt"
        
        # Copy image
        shutil.copy(os.path.join(images_dir, img_name), f"{base_dir}/images/{split}/{img_name}")
        
        # Copy corresponding label
        label_path = os.path.join(labels_dir, label_file)
        if os.path.exists(label_path):
            shutil.copy(label_path, f"{base_dir}/labels/{split}/{label_file}")

In [4]:
data_yaml = """
path: /kaggle/working/dataset
train: images/train
val: images/val

nc: 1
names: ['disease']
"""

with open("/kaggle/working/data.yaml", "w") as f:
    f.write(data_yaml)


In [5]:
from ultralytics import YOLO

model = YOLO("yolov8s-seg.pt")  # Use 'yolov8n-seg.pt' for faster training

model.train(
    data="/kaggle/working/data.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    task="segment"
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 22.8M/22.8M [00:00<00:00, 176MB/s]


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine

100%|██████████| 755k/755k [00:00<00:00, 15.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:00<00:00,  3.89it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.45it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50         95       0.81      0.896      0.907      0.657      0.836      0.749      0.806      0.482
Speed: 0.2ms preprocess, 9.5ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to runs/segment/train


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e5270de7550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [6]:
results = model.predict(source="/kaggle/working/dataset/images/val", save=True)


image 1/50 /kaggle/working/dataset/images/val/disease10_aug2.jpg: 640x640 1 disease, 19.5ms
image 2/50 /kaggle/working/dataset/images/val/disease12_aug3.jpg: 640x640 1 disease, 19.5ms
image 3/50 /kaggle/working/dataset/images/val/disease12_aug4.jpg: 640x640 1 disease, 19.5ms
image 4/50 /kaggle/working/dataset/images/val/disease14_aug0.jpg: 640x640 4 diseases, 19.5ms
image 5/50 /kaggle/working/dataset/images/val/disease14_aug3.jpg: 640x640 4 diseases, 19.5ms
image 6/50 /kaggle/working/dataset/images/val/disease14_aug4.jpg: 640x640 5 diseases, 19.5ms
image 7/50 /kaggle/working/dataset/images/val/disease15_aug1.jpg: 640x640 2 diseases, 19.5ms
image 8/50 /kaggle/working/dataset/images/val/disease16_aug1.jpg: 640x640 1 disease, 13.8ms
image 9/50 /kaggle/working/dataset/images/val/disease16_aug2.jpg: 640x640 1 disease, 13.8ms
image 10/50 /kaggle/working/dataset/images/val/disease18_aug1.jpg: 640x640 3 diseases, 13.8ms
image 11/50 /kaggle/working/dataset/images/val/disease18_aug2.jpg: 640x64

In [7]:
results = model.predict(source="/kaggle/working/dataset/images/val", save=True)


image 1/50 /kaggle/working/dataset/images/val/disease10_aug2.jpg: 640x640 1 disease, 9.6ms
image 2/50 /kaggle/working/dataset/images/val/disease12_aug3.jpg: 640x640 1 disease, 9.6ms
image 3/50 /kaggle/working/dataset/images/val/disease12_aug4.jpg: 640x640 1 disease, 9.8ms
image 4/50 /kaggle/working/dataset/images/val/disease14_aug0.jpg: 640x640 4 diseases, 9.9ms
image 5/50 /kaggle/working/dataset/images/val/disease14_aug3.jpg: 640x640 4 diseases, 10.4ms
image 6/50 /kaggle/working/dataset/images/val/disease14_aug4.jpg: 640x640 5 diseases, 10.5ms
image 7/50 /kaggle/working/dataset/images/val/disease15_aug1.jpg: 640x640 2 diseases, 10.7ms
image 8/50 /kaggle/working/dataset/images/val/disease16_aug1.jpg: 640x640 1 disease, 10.9ms
image 9/50 /kaggle/working/dataset/images/val/disease16_aug2.jpg: 640x640 1 disease, 11.2ms
image 10/50 /kaggle/working/dataset/images/val/disease18_aug1.jpg: 640x640 3 diseases, 11.2ms
image 11/50 /kaggle/working/dataset/images/val/disease18_aug2.jpg: 640x640 4 

In [8]:
import shutil
import os

# Delete everything inside /kaggle/working/
for filename in os.listdir("/kaggle/working"):
    file_path = os.path.join("/kaggle/working", filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file or link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directory
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')


In [9]:
results = model.predict(source="/kaggle/input/picture", save=True)


image 1/1 /kaggle/input/picture/pexels-kelly-1179532-2559931.jpg: 640x448 1 disease, 53.1ms
Speed: 3.2ms preprocess, 53.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 448)
Results saved to runs/segment/train


In [10]:
results = model.predict(source="/kaggle/input/picturedis", save=True)


image 1/1 /kaggle/input/picturedis/pic1.jpeg: 480x640 1 disease, 54.6ms
Speed: 2.4ms preprocess, 54.6ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/segment/train2


In [11]:
results = model.predict(source="/kaggle/input/piccc2", save=True)


image 1/1 /kaggle/input/piccc2/pic2.jpeg: 480x640 1 disease, 16.8ms
Speed: 1.8ms preprocess, 16.8ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/segment/train3
